In [4]:
from typing import List
from functools import wraps

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import yfinance as yf
import datetime as dt

jtplot.style(figsize=(15, 9))

In [2]:
# tickers = ['GLD', 'GDX', 'AAPL', 'SPY']
# # tickers = ['MSFT', 'NVDA', 'AMD', 'META']
# d_start = dt.datetime(2010, 1, 1)
# d_final = dt.datetime(2020, 12, 31)
#  period = '1d'

# df = yf.download(tickers, start=d_start, end=d_final, period=period, auto_adjust=True)
# df.to_csv(f"../../../data/bt/{'_'.join(tickers)}__{period.upper()}.csv")

In [5]:
import os

DATA_PATH = '../../data/bt/'

def read_csv_bt(csv:str)->pd.DataFrame:
    file_path = f"{DATA_PATH}{csv}"
    tickers, _ = csv.split('__')
    if len(tickers)>1:
        return pd.read_csv(file_path, parse_dates=True, header=[0, 1], index_col=0)['Close']
    else:
        return pd.read_csv(file_path, parse_dates=True, index_col=0)['Close']

def get_backtest_data()->List[list]:
    bt_files = os.listdir(DATA_PATH)
    csv_files = []
    for csv in bt_files:
        try:
            tickers, period = csv.split('__')
            df = read_csv_bt(csv) # send to some dict with data???
            csv_files.append({
                'csv': csv,
                'tickers': list(map(str.upper, tickers.split('_'))),
                'period': period.replace('.csv', ''),
                'start': df.iloc[0].name.strftime('%d/%m/%Y'),
                'end': df.iloc[-1].name.strftime('%d/%m/%Y')
            })
        except:
            continue
    return csv_files


## Strategies for testing

In [6]:
class Strategy:
    def __init__(self):
        pass
    
    def set_data(self, ticker:str, data:pd.DataFrame):
        self.data = data.copy()
        self.data['returns'] = np.log(self.data[ticker] / self.data[ticker].shift(1))
        self.ticker = ticker
        self.strategy = []
        self.drawdown = []
    
    def get_result(self):
        strategy_result = self.data[self.strategy+['returns']].sum().apply(np.exp)
        drawdown_result = self.data[self.drawdown].max()
        result = pd.concat([strategy_result, drawdown_result]).to_frame()
        result.rename(columns={0: self.ticker}, inplace=True)
        
        return result


In [7]:
class SMA(Strategy):
    def __init__(self, sma1:List[int]=[42], sma2:List[int]=[252]):
        self.sma1 = sma1
        self.sma2 = sma2
    
    def run_strategy(self):
        for s1, s2 in zip(self.sma1, self.sma2):
            self.data[f'SMA1_{s1}'] = self.data[self.ticker].rolling(s1).mean()
            self.data[f'SMA2_{s2}'] = self.data[self.ticker].rolling(s2).mean()
            self.data.dropna(inplace=True)

            self.data[f'position_{s1}_{s2}'] = np.where(self.data[f'SMA1_{s1}'] > self.data[f'SMA2_{s2}'], 1, -1)
            self.data[f'strategy_{s1}_{s2}'] = self.data[f'position_{s1}_{s2}'].shift(1) * self.data['returns']
            self.strategy.append(f'strategy_{s1}_{s2}')

            self.data[f'cumret_{s1}_{s2}'] = self.data[f'strategy_{s1}_{s2}'].cumsum().apply(np.exp)
            self.data[f'cummax_{s1}_{s2}'] = self.data[f'cumret_{s1}_{s2}'].cummax()
            self.data[f'drawdown_{s1}_{s2}'] = self.data[f'cummax_{s1}_{s2}'] - self.data[f'cumret_{s1}_{s2}']
            self.drawdown.append(f'drawdown_{s1}_{s2}')

        result = self.get_result()
    
        return result, self.data.copy()
    

In [8]:
class MOM(Strategy):
    def __init__(self, moms:List[int]=[1]):
        self.moms = moms
    
    def run_strategy(self):
        for mom in self.moms:
            self.data[f'MOM_{mom}'] = self.data[self.ticker].rolling(mom).mean()
            self.data.dropna(inplace=True)

            self.data[f'position_{mom}'] = np.sign(self.data['returns'].rolling(mom).mean())
            self.data[f'strategy_{mom}'] = self.data[f'position_{mom}'].shift(1) * self.data['returns']
            self.strategy.append(f'strategy_{mom}')

            self.data[f'cumret_{mom}'] = self.data[f'strategy_{mom}'].cumsum().apply(np.exp)
            self.data[f'cummax_{mom}'] = self.data[f'cumret_{mom}'].cummax()
            self.data[f'drawdown_{mom}'] = self.data[f'cummax_{mom}'] - self.data[f'cumret_{mom}']
            self.drawdown.append(f'drawdown_{mom}')

        result = self.get_result()
    
        return result, self.data.copy()


In [9]:
class MeanRe(Strategy):
    def __init__(self, sma:List[int]=[25], threshold:float=3.5):
        self.sma = sma
        self.threshold = threshold
        
    def run_strategy(self):
        for s in self.sma:
            self.data['SMA'] = self.data[self.ticker].rolling(s).mean()
            self.data['distance'] = self.data[self.ticker] - self.data['SMA']
            self.data['distance'].dropna(inplace=True)
            
            self.data[f'position_{s}'] = np.where(self.data['distance'] > self.threshold,
                                                  -1, np.nan)  
            self.data[f'position_{s}'] = np.where(self.data['distance'] < -self.threshold,
                                                  1, self.data[f'position_{s}'])  
            self.data[f'position_{s}'] = np.where(self.data['distance'] * self.data['distance'].shift(1) < 0,
                                                  0, self.data[f'position_{s}'])  
            self.data[f'position_{s}'] = self.data[f'position_{s}'].ffill().fillna(0)
            self.data[f'strategy_{s}'] = self.data[f'position_{s}'].shift(1) * self.data['returns']
            self.strategy.append(f'strategy_{s}')
            
            self.data[f'cumret_{s}'] = self.data[f'strategy_{s}'].cumsum().apply(np.exp)
            self.data[f'cummax_{s}'] = self.data[f'cumret_{s}'].cummax()
            self.data[f'drawdown_{s}'] = self.data[f'cummax_{s}'] - self.data[f'cumret_{s}']
            self.drawdown.append(f'drawdown_{s}')
            
        result = self.get_result()
        
        return result, self.data.copy()
    

In [11]:
def start_backtesting(strategy_cls, *args)->list:
    bt_results_dfs = []
    bt_data_files = get_backtest_data()
    for bt in bt_data_files:
        csv, tickers, period, _, _ = bt.values()
        data = read_csv_bt(csv)

        single_ticker_result = []
        STRATEGY = get_strategy(strategy_cls, *args)
        for ticker in tickers:
            STRATEGY.set_data(ticker, pd.DataFrame(data[ticker]))
            result, full_data = STRATEGY.run_strategy()
            
            single_ticker_result.append(result)

        single_ticker_result = pd.concat(single_ticker_result, axis=1)
        single_ticker_result.index.name = period
        bt_results_dfs.append(single_ticker_result)
    return bt_results_dfs
        
def get_strategy(strategy, *args):
    strategies = dict(
        sma=SMA,
        mom=MOM,
        mean_re=MeanRe,
    )
    return strategies[strategy](*args)


In [10]:
SMA1 = [42, 24, 18, 6]
SMA2 = [252, 180, 64, 22]
start_backtesting('sma', SMA1, SMA2)

[                      GLD       GDX       AAPL       SPY
 1D                                                      
 strategy_42_252  1.308799  1.065529   6.424632  1.466223
 strategy_24_180  1.162980  1.102915   6.462446  1.668211
 strategy_18_64   1.611339  0.746799   3.299714  1.804156
 strategy_6_22    1.462546  2.269095   5.655584  0.803092
 returns          1.107165  0.732320  10.219678  3.437358
 drawdown_42_252  0.447434  1.041484   4.107096  0.623526
 drawdown_24_180  0.402130  0.787659   1.729420  1.023388
 drawdown_18_64   0.308936  1.408235   1.144106  0.520338
 drawdown_6_22    0.322727  2.556176   1.345458  0.557455,
                       GLD       GDX      AAPL       SPY
 1H                                                     
 strategy_42_252  1.210578  0.547866  1.141445  1.205247
 strategy_24_180  1.001059  0.695432  1.112150  1.183577
 strategy_18_64   1.258914  1.151021  1.049129  1.315175
 strategy_6_22    0.788741  0.802870  0.819486  1.026587
 returns          1

In [77]:
start_backtesting('mom', [10])

[                  GLD       GDX       AAPL       SPY
 1D                                                  
 strategy_10  0.995737  0.314059   3.301410  1.175725
 returns      1.586182  0.809055  20.724333  4.027025
 drawdown_10  0.323483  1.662379   1.366986  0.677924,
                   GLD       GDX      AAPL       SPY
 1H                                                 
 strategy_10  1.011674  0.624452  0.577015  0.893413
 returns      1.565266  1.620833  1.586997  1.577566
 drawdown_10  0.159304  0.511475  0.485228  0.169165,
                  MSFT       NVDA        AMD      META
 1D                                                   
 strategy_10  0.565772   0.743711   2.656222  0.325733
 returns      9.193323  32.440119  10.254445  9.184797
 drawdown_10  1.026777   1.593146   8.999155  1.683596,
                  MSFT       NVDA        AMD      META
 1H                                                   
 strategy_10  0.538790   0.885990   2.656222  0.325733
 returns      9.193321

In [10]:
start_backtesting('mean_re')

[                  GLD       GDX       AAPL       SPY
 1D                                                  
 strategy_25  1.095107  3.003503   0.414385  1.214852
 returns      1.618397  0.824147  20.280812  4.083878
 drawdown_25  0.297397  0.844094   0.832532  0.747674,
                   GLD       GDX      AAPL       SPY
 1H                                                 
 strategy_25  0.996823  1.000000  1.092354  1.124085
 returns      1.538725  1.591653  1.594131  1.570848
 drawdown_25  0.114540  0.000000  0.248219  0.156182,
                  MSFT       NVDA       AMD      META
 1D                                                  
 strategy_25  1.706064   1.000000  0.952347  1.240384
 returns      9.196292  30.931284  9.514433  7.111431
 drawdown_25  0.717983   0.000000  0.541103  0.846389,
                  MSFT       NVDA       AMD      META
 1H                                                  
 strategy_25  1.706061   1.000000  0.952347  1.240384
 returns      9.196294  30.931